In [1]:
import alminer
import pandas as pd
import numpy as np
import nltk
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from textparser import TextParser
from ast import literal_eval

## Text processing functions NEEDS REVIEW OF REGEX PARSING FOR TRANSPARENCY

In [2]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text = text.strip()  
    text = re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text
 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [3]:
test_sent = 'Hello, this is a tests concerning H2O = H2 + O. What happens if we trying this?'

### `preprocess` removes punctuation NEEDS REVIEW OF REGEX

In [4]:
preprocessed_test = preprocess(test_sent)
preprocessed_test

'hello this is a tests concerning h o h o what happens if we trying this'

### `stopword` removes stopwords

In [5]:
sw_removed_test = stopword(preprocessed_test)
sw_removed_test

'hello tests concerning h h happens trying'

### `lemmatizer` lemmatizes text

In [6]:
lemma_test = lemmatizer(sw_removed_test)
lemma_test

'hello test concern h h happen try'

## Import data from ALMA data explorer and AlMiner

### Read exported observation data from ALMA tool
* There is a row in the data that gets split on a weird character or something that I changed manually

In [7]:
observations = pd.read_csv('observations.csv')
#observations = observations.set_index('Project code')
observations

/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_46852/3868851997.py:1: DtypeWarning: Columns (4,36,39) have mixed types. Specify dtype option on import or set low_memory=False.
  observations = pd.read_csv('observations.csv')


,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,...,PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections
0,2011.0.00001.E,GRB110715A,237.683625,-46.235833,7,0.248443,"[337.02..339.01GHz,31250.00kHz,6.33mJy/beam@10...",2018-07-10,2,0.526604,...,0.581143,NaN,uid://A002/X24c117/Xe4,1,Proof of Concept of Response to Targets of Opp...,E,NaN,TARGET WVR,PASS,NaN
1,2011.0.00001.SV,TW Hya,165.466276,-34.704728,7,0.368748,"[343.51..343.98GHz,244.14kHz,5.17mJy/beam@10km...",2016-06-24,8,1.670760,...,1.323529,NaN,uid://A002/X1d4c10/X53,3,Science verification observation of TW Hya,SV,NaN,TARGET,PASS,NaN
2,2011.0.00002.E,J1820-2528,275.241036,-25.470162,7,0.052254,"[335.00..336.98GHz,31250.00kHz,1.44mJy/beam@10...",2018-07-10,0,0.314430,...,0.968139,uid://A002/X845868/X39,uid://A002/X845868/X3a,2,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET,PASS,NaN
3,2011.0.00002.E,Ephemeris,282.645671,-20.330943,7,0.036949,"[334.99..336.98GHz,31250.00kHz,1.02mJy/beam@10...",2018-07-10,0,0.302198,...,0.968139,uid://A002/X845868/X39,uid://A002/X845868/X3a,2,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET,PASS,NaN
4,2011.0.00002.E,J1820-254,275.241036,-25.470162,7,0.077639,"[335.01..336.99GHz,31250.00kHz,2.14mJy/beam@10...",2018-07-10,0,0.342764,...,1.356428,uid://A002/X7028c3/X73,uid://A002/X7028c3/X74,1,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET WVR,PASS,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68979,2023.A.00003.S,COS-z12-1,149.730053,2.121326,6,0.015278,"[241.69..243.56GHz,7812.01kHz,0.46mJy/beam@10k...",2024-07-10,0,0.311051,...,0.639321,uid://A001/X362b/Xecb,uid://A001/X362b/Xecc,2,[OIII] Confirmation for Intrinsically Luminous...,S,NaN,TARGET,PASS,NaN
68980,2023.A.00003.S,COS-z12-1,149.730053,2.121326,6,0.022838,"[255.32..257.18GHz,7812.01kHz,0.65mJy/beam@10k...",2024-07-03,0,0.396969,...,1.733926,uid://A001/X362b/Xed7,uid://A001/X362b/Xed8,2,[OIII] Confirmation for Intrinsically Luminous...,S,NaN,TARGET,PASS,NaN
68981,2023.A.00003.S,COS-z12-1,149.730053,2.121326,6,0.015852,"[245.10..246.96GHz,7812.01kHz,0.48mJy/beam@10k...",2024-07-10,0,0.326002,...,0.648661,uid://A001/X362b/Xedb,uid://A001/X362b/Xedc,2,[OIII] Confirmation for Intrinsically Luminous...,S,NaN,TARGET,PASS,NaN
68982,2023.A.00009.S,HOPS-315,86.515125,-0.247083,7,0.030525,"[344.07..344.53GHz,1128.91kHz,0.61mJy/beam@10k...",NaN,0,0.034967,...,0.563973,uid://A001/X362e/X236,uid://A001/X362e/X237,4,Finding t=0: Tracing the Origins of Rocky Plan...,S,NaN,TARGET,UNKNOWN,NaN


In [8]:
test = observations.loc[observations['Project code'] == '2011.0.00001.E']
test['Frequency support'].values

array(['[337.02..339.01GHz,31250.00kHz,6.33mJy/beam@10km/s,0.48uJy/beam@native, XX YY,continuum] U [338.96..340.94GHz,31250.00kHz,6.13mJy/beam@10km/s,0.46uJy/beam@native, XX YY,continuum] U [349.02..351.01GHz,31250.00kHz,6.41mJy/beam@10km/s,0.49uJy/beam@native, XX YY,continuum] U [351.02..353.01GHz,31250.00kHz,7.66mJy/beam@10km/s,0.59uJy/beam@native, XX YY,continuum]'],
      dtype=object)

### Query with ALminer to get proposal IDs and abstracts
* Needs publication year for some reason?

In [9]:
query_str = """
            SELECT DISTINCT publication_year,
                proposal_id,
                proposal_abstract
            FROM ivoa.obscore
            ORDER BY proposal_id
            """

In [10]:
query = alminer.run_query(query_str)

In [11]:
id_abs = alminer.explore(query, allcols=True, allrows=False)
id_abs = id_abs.drop(['publication_year'], axis = 1)
id_abs

,proposal_id,proposal_abstract
0,2011.0.00001.E,ALMA engineering data release.
1,2011.0.00001.SV,Science Verification (SV) is the process by wh...
2,2011.0.00002.E,ALMA engineering data release.
3,2011.0.00002.SV,Science Verification (SV) is the process by wh...
4,2011.0.00003.E,ALMA engineering data release.
5,2011.0.00003.SV,Science Verification (SV) is the process by wh...
6,2011.0.00004.E,ALMA engineering data release.
7,2011.0.00004.SV,Science Verification (SV) is the process by wh...
8,2011.0.00005.E,ALMA engineering data release.
...,...,...


### Merge data frames on Project code

In [12]:
joined = observations.merge(id_abs, how = 'left', right_on = 'proposal_id', left_on = 'Project code')
joined

,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,Min.vel.res.,Array,Mosaic,Max.reco.scale,FOV,Scientific category,Science keyword,Int.Time,Gal.lon.,Gal.lat.,Min.freq.res.,Pol products,L5 BL,L80 BL,Obs.date,End date,PI name,SB name,Proposal authors,Line sens.(10 km/s),PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections,proposal_id,proposal_abstract
0,2011.0.00001.E,GRB110715A,237.683625,-46.235833,7,0.248443,"[337.02..339.01GHz,31250.00kHz,6.33mJy/beam@10...",2018-07-10,2,0.526604,26.540696,12m,NaN,6.001451,16.877382,NaN,NaN,1225.728,332.300705,6.160732,31251.848549,XX YY,29.356560,195.359850,2011-07-19,2011-07-19,"observatory, ALMA",GRB110715A,NaN,6.126660,0.581143,NaN,uid://A002/X24c117/Xe4,1,Proof of Concept of Response to Targets of Opp...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00001.E,ALMA engineering data release.
1,2011.0.00001.SV,TW Hya,165.466276,-34.704728,7,0.368748,"[343.51..343.98GHz,244.14kHz,5.17mJy/beam@10km...",2016-06-24,8,1.670760,0.204330,12m,NaN,10.917078,16.595796,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",7106.400,278.678757,22.954625,244.157214,XX YY,18.668068,59.099155,2011-04-22,2011-04-22,"observatory, ALMA","TW Hya: Band 7: CO, HCO+, HC15N",NaN,4.512420,1.323529,NaN,uid://A002/X1d4c10/X53,3,Science verification observation of TW Hya,SV,NaN,TARGET,PASS,NaN,2011.0.00001.SV,Science Verification (SV) is the process by wh...
2,2011.0.00002.E,J1820-2528,275.241036,-25.470162,7,0.052254,"[335.00..336.98GHz,31250.00kHz,1.44mJy/beam@10...",2018-07-10,0,0.314430,26.691931,12m,NaN,2.592821,16.977021,NaN,NaN,2419.200,6.887193,-5.138818,31249.590050,XX YY,70.123490,322.764370,2014-07-11,2014-07-15,"observatory, ALMA",Pluto,NaN,1.261645,0.968139,uid://A002/X845868/X39,uid://A002/X845868/X3a,2,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET,PASS,NaN,2011.0.00002.E,ALMA engineering data release.
3,2011.0.00002.E,Ephemeris,282.645671,-20.330943,7,0.036949,"[334.99..336.98GHz,31250.00kHz,1.02mJy/beam@10...",2018-07-10,0,0.302198,26.691931,12m,mosaic,2.496266,366.254737,NaN,NaN,2419.200,14.593289,-8.935012,31249.091863,XX YY,73.160640,334.674040,2014-07-11,2014-07-15,"observatory, ALMA",Pluto,NaN,0.892122,0.968139,uid://A002/X845868/X39,uid://A002/X845868/X3a,2,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET,PASS,NaN,2011.0.00002.E,ALMA engineering data release.
4,2011.0.00002.E,J1820-254,275.241036,-25.470162,7,0.077639,"[335.01..336.99GHz,31250.00kHz,2.14mJy/beam@10...",2018-07-10,0,0.342764,26.691931,12m 7m,NaN,7.162272,16.976557,NaN,NaN,1088.640,6.887193,-5.138818,31250.443204,XX YY,24.743156,301.904570,2013-11-25,2013-11-25,"observatory, ALMA",Pluto,NaN,1.884315,1.356428,uid://A002/X7028c3/X73,uid://A002/X7028c3/X74,1,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00002.E,ALMA engineering data release.
5,2011.0.00002.E,J1923-210,290.884124,-21.075926,7,0.073898,"[334.99..336.98GHz,31250.00kHz,2.03mJy/beam@10...",2018-07-10,0,0.290913,26.691931,12m,NaN,2.344089,16.977464,NaN,NaN,1209.600,17.178427,-16.254503,31248.774306,XX YY,78.169900,348.290950,2014-07-11,2014-07-15,"observatory, ALMA",Pluto,NaN,1.784248,0.968139,uid://A002/X845868/X39,uid://A002/X845868/X3a,2,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET,PASS,NaN,2011.0.00002.E,ALMA engineering data release.
6,2011.0.00002.E,J1924-2105,290.884124,-21.075926,7,0.087444,"[334.96..337.00GHz,31250.00kHz,2.39mJy/beam@10...",2018-07-10,0,0.330796,26.691931,12m 7m,NaN,6.524738,16.979130,NaN,NaN,1753.920,17.178427,-16.254503,31245.708731,XX YY,44.483486,248.209050,2013-11-25,2014-04-22,"observatory, ALMA",Pluto,NaN,2.122659,1.146463,uid://A002/X7028c3/X73,uid://A002/X7028c3/X74,3,Pluto Band 7 TDM data in support of NASA JPL N...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00002.E,ALMA engineering data release.
7,2011.0.00002.E,Ephemeris,282.993852,-20.160041,7,0.04

### Double check there are no mismatched rows in join

In [13]:
joined.loc[~(joined['Project code'] == joined['proposal_id'])]

,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,Min.vel.res.,Array,Mosaic,Max.reco.scale,FOV,Scientific category,Science keyword,Int.Time,Gal.lon.,Gal.lat.,Min.freq.res.,Pol products,L5 BL,L80 BL,Obs.date,End date,PI name,SB name,Proposal authors,Line sens.(10 km/s),PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections,proposal_id,proposal_abstract


### Inspect a row or two

In [14]:
row_proj = joined.loc[joined['Project code'] == '2011.0.00007.SV']
row_proj

,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,Min.vel.res.,Array,Mosaic,Max.reco.scale,FOV,Scientific category,Science keyword,Int.Time,Gal.lon.,Gal.lat.,Min.freq.res.,Pol products,L5 BL,L80 BL,Obs.date,End date,PI name,SB name,Proposal authors,Line sens.(10 km/s),PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections,proposal_id,proposal_abstract
23,2011.0.00007.SV,IRAS16293-2422-a,248.094732,-24.475694,9,2.898375,"[686.49..688.37GHz,976.56kHz,52.20mJy/beam@10k...",NaN,13,0.170097,0.415161,12m,mosaic,1.740436,15.368732,ISM and star formation,"Low-mass star formation, Astrochemistry",1964.861,353.936013,15.840312,976.461231,XX YY,68.867810,274.37830,2012-04-16,2012-04-17,"observatory, ALMA",IRAS16293-2422 - B9 - 1924-292 Bp - POST-TRANS...,NaN,51.980702,0.422559,NaN,uid://A002/X3cd6b2/X59,4,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
24,2011.0.00007.SV,IRAS16293-2422,248.094732,-24.475694,4,0.017913,"[145.11..146.99GHz,976.56kHz,0.56mJy/beam@10km...",NaN,13,0.733760,0.248550,12m,NaN,5.914166,38.274008,ISM and star formation,"Low-mass star formation, Astrochemistry",3659.040,353.936013,15.840312,122.073607,XX YY,67.556435,317.95435,2014-07-11,2014-07-11,"observatory, ALMA",IRAS1629_a_04_TE,NaN,0.509668,1.100789,uid://A002/X8666bf/X13,uid://A002/X8666bf/X14,1,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...


#### Check out frequency support for selected project

In [15]:
type(row_proj['Frequency support'].iloc[0])

str

#### Frequency support is a string?! Let's see how it's formatted

In [16]:
row_proj['Frequency support'].iloc[0]

'[686.49..688.37GHz,976.56kHz,52.20mJy/beam@10km/s,5.77uJy/beam@native, XX YY,line] U [688.49..690.37GHz,976.56kHz,51.98mJy/beam@10km/s,5.76uJy/beam@native, XX YY,line] U [690.29..692.17GHz,976.56kHz,52.05mJy/beam@10km/s,5.77uJy/beam@native, XX YY,line] U [703.24..705.12GHz,976.56kHz,52.61mJy/beam@10km/s,5.89uJy/beam@native, XX YY,line]'

#### Frequency support string looks like it could be made a list of lists if we give it some work
* We can replace `' U '` with a comma
* We will need to make sure the entries in each frequency support list are strings by putting `'` quotes around them
* We can add brackets to the beginning and end of the string to make everything in there a list of lists

In [17]:
test = joined.loc[joined['Project code'] == '2011.0.00007.SV']
test['Frequency support'] = test['Frequency support'].str.replace(' U ', ';')
test['Frequency support'] = test['Frequency support'].str.replace(',', '\',\'')
test['Frequency support'] = test['Frequency support'].str.replace('[', '[\'')
test['Frequency support'] = test['Frequency support'].str.replace(']', '\']')
test['Frequency support'] = test['Frequency support'].str.replace(';', ',')
test['Frequency support'] = '[' + test['Frequency support'] + ']'
test

/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_46852/2395481783.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Frequency support'] = test['Frequency support'].str.replace(' U ', ';')
/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_46852/2395481783.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Frequency support'] = test['Frequency support'].str.replace(',', '\',\'')
/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_46852/2395481783.py:4: SettingWithCopyW

,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,Min.vel.res.,Array,Mosaic,Max.reco.scale,FOV,Scientific category,Science keyword,Int.Time,Gal.lon.,Gal.lat.,Min.freq.res.,Pol products,L5 BL,L80 BL,Obs.date,End date,PI name,SB name,Proposal authors,Line sens.(10 km/s),PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections,proposal_id,proposal_abstract
23,2011.0.00007.SV,IRAS16293-2422-a,248.094732,-24.475694,9,2.898375,"[['686.49..688.37GHz','976.56kHz','52.20mJy/be...",NaN,13,0.170097,0.415161,12m,mosaic,1.740436,15.368732,ISM and star formation,"Low-mass star formation, Astrochemistry",1964.861,353.936013,15.840312,976.461231,XX YY,68.867810,274.37830,2012-04-16,2012-04-17,"observatory, ALMA",IRAS16293-2422 - B9 - 1924-292 Bp - POST-TRANS...,NaN,51.980702,0.422559,NaN,uid://A002/X3cd6b2/X59,4,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
24,2011.0.00007.SV,IRAS16293-2422,248.094732,-24.475694,4,0.017913,"[['145.11..146.99GHz','976.56kHz','0.56mJy/bea...",NaN,13,0.733760,0.248550,12m,NaN,5.914166,38.274008,ISM and star formation,"Low-mass star formation, Astrochemistry",3659.040,353.936013,15.840312,122.073607,XX YY,67.556435,317.95435,2014-07-11,2014-07-11,"observatory, ALMA",IRAS1629_a_04_TE,NaN,0.509668,1.100789,uid://A002/X8666bf/X13,uid://A002/X8666bf/X14,1,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...


### It looks like Frequency support is formatted as a string representing a list of lists
* We can use `literal eval` to turn this into an actual list of lists

In [18]:
test['Frequency support'] = test['Frequency support'].apply(literal_eval)

/var/folders/vx/9pb2fpkj3m5bkbwxzyx6sz5m0000gn/T/ipykernel_46852/1944917856.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['Frequency support'] = test['Frequency support'].apply(literal_eval)


#### Now that Frequency support is a list of lists we can use explode to make rows for each Frequency support list of lists by observation

In [19]:
test.explode('Frequency support')

,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,Min.vel.res.,Array,Mosaic,Max.reco.scale,FOV,Scientific category,Science keyword,Int.Time,Gal.lon.,Gal.lat.,Min.freq.res.,Pol products,L5 BL,L80 BL,Obs.date,End date,PI name,SB name,Proposal authors,Line sens.(10 km/s),PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections,proposal_id,proposal_abstract
23,2011.0.00007.SV,IRAS16293-2422-a,248.094732,-24.475694,9,2.898375,"[686.49..688.37GHz, 976.56kHz, 52.20mJy/beam@1...",NaN,13,0.170097,0.415161,12m,mosaic,1.740436,15.368732,ISM and star formation,"Low-mass star formation, Astrochemistry",1964.861,353.936013,15.840312,976.461231,XX YY,68.867810,274.37830,2012-04-16,2012-04-17,"observatory, ALMA",IRAS16293-2422 - B9 - 1924-292 Bp - POST-TRANS...,NaN,51.980702,0.422559,NaN,uid://A002/X3cd6b2/X59,4,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
23,2011.0.00007.SV,IRAS16293-2422-a,248.094732,-24.475694,9,2.898375,"[688.49..690.37GHz, 976.56kHz, 51.98mJy/beam@1...",NaN,13,0.170097,0.415161,12m,mosaic,1.740436,15.368732,ISM and star formation,"Low-mass star formation, Astrochemistry",1964.861,353.936013,15.840312,976.461231,XX YY,68.867810,274.37830,2012-04-16,2012-04-17,"observatory, ALMA",IRAS16293-2422 - B9 - 1924-292 Bp - POST-TRANS...,NaN,51.980702,0.422559,NaN,uid://A002/X3cd6b2/X59,4,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
23,2011.0.00007.SV,IRAS16293-2422-a,248.094732,-24.475694,9,2.898375,"[690.29..692.17GHz, 976.56kHz, 52.05mJy/beam@1...",NaN,13,0.170097,0.415161,12m,mosaic,1.740436,15.368732,ISM and star formation,"Low-mass star formation, Astrochemistry",1964.861,353.936013,15.840312,976.461231,XX YY,68.867810,274.37830,2012-04-16,2012-04-17,"observatory, ALMA",IRAS16293-2422 - B9 - 1924-292 Bp - POST-TRANS...,NaN,51.980702,0.422559,NaN,uid://A002/X3cd6b2/X59,4,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
23,2011.0.00007.SV,IRAS16293-2422-a,248.094732,-24.475694,9,2.898375,"[703.24..705.12GHz, 976.56kHz, 52.61mJy/beam@1...",NaN,13,0.170097,0.415161,12m,mosaic,1.740436,15.368732,ISM and star formation,"Low-mass star formation, Astrochemistry",1964.861,353.936013,15.840312,976.461231,XX YY,68.867810,274.37830,2012-04-16,2012-04-17,"observatory, ALMA",IRAS16293-2422 - B9 - 1924-292 Bp - POST-TRANS...,NaN,51.980702,0.422559,NaN,uid://A002/X3cd6b2/X59,4,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
24,2011.0.00007.SV,IRAS16293-2422,248.094732,-24.475694,4,0.017913,"[145.11..146.99GHz, 976.56kHz, 0.56mJy/beam@10...",NaN,13,0.733760,0.248550,12m,NaN,5.914166,38.274008,ISM and star formation,"Low-mass star formation, Astrochemistry",3659.040,353.936013,15.840312,122.073607,XX YY,67.556435,317.95435,2014-07-11,2014-07-11,"observatory, ALMA",IRAS1629_a_04_TE,NaN,0.509668,1.100789,uid://A002/X8666bf/X13,uid://A002/X8666bf/X14,1,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
24,2011.0.00007.SV,IRAS16293-2422,248.094732,-24.475694,4,0.017913,"[147.01..147.24GHz, 122.07kHz, 0.53mJy/beam@10...",NaN,13,0.733760,0.248550,12m,NaN,5.914166,38.274008,ISM and star formation,"Low-mass star formation, Astrochemistry",3659.040,353.936013,15.840312,122.073607,XX YY,67.556435,317.95435,2014-07-11,2014-07-11,"observatory, ALMA",IRAS1629_a_04_TE,NaN,0.509668,1.100789,uid://A002/X8666bf/X13,uid://A002/X8666bf/X14,1,Science verification observation of IRAS16293,SV,NaN,TARGET,UNKNOWN,NaN,2011.0.00007.SV,Science Verification (SV) is the process by wh...
24,2011.0.00007.SV,IRAS16293-2422,248.

#### Looks like it worked! We can apply it to joined to get lists for each of the Frequency support entries by observation

In [20]:
joined['Frequency support'] = joined['Frequency support'].str.replace(' U ', ';')
joined['Frequency support'] = joined['Frequency support'].str.replace(',', '\',\'')
joined['Frequency support'] = joined['Frequency support'].str.replace('[', '[\'')
joined['Frequency support'] = joined['Frequency support'].str.replace(']', '\']')
joined['Frequency support'] = joined['Frequency support'].str.replace(';', ',')
joined['Frequency support'] = '[' + joined['Frequency support'] + ']'

In [21]:
joined['Frequency support'] = joined['Frequency support'].apply(literal_eval)

In [22]:
full = joined.explode('Frequency support').reset_index().drop('index', axis = 1)
full

,Project code,ALMA source name,RA,Dec,Band,Cont.sens.,Frequency support,Release date,Publications,Ang.res.,Min.vel.res.,Array,Mosaic,Max.reco.scale,FOV,Scientific category,Science keyword,Int.Time,Gal.lon.,Gal.lat.,Min.freq.res.,Pol products,L5 BL,L80 BL,Obs.date,End date,PI name,SB name,Proposal authors,Line sens.(10 km/s),PWV,Group ous id,Member ous id,Asdm,Project title,Type,Joint proposals,Scan intent,QA2 Status,Collections,proposal_id,proposal_abstract
0,2011.0.00001.E,GRB110715A,237.683625,-46.235833,7,0.248443,"[337.02..339.01GHz, 31250.00kHz, 6.33mJy/beam@...",2018-07-10,2,0.526604,26.540696,12m,NaN,6.001451,16.877382,NaN,NaN,1225.728,332.300705,6.160732,31251.848549,XX YY,29.356560,195.359850,2011-07-19,2011-07-19,"observatory, ALMA",GRB110715A,NaN,6.126660,0.581143,NaN,uid://A002/X24c117/Xe4,1,Proof of Concept of Response to Targets of Opp...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00001.E,ALMA engineering data release.
1,2011.0.00001.E,GRB110715A,237.683625,-46.235833,7,0.248443,"[338.96..340.94GHz, 31250.00kHz, 6.13mJy/beam@...",2018-07-10,2,0.526604,26.540696,12m,NaN,6.001451,16.877382,NaN,NaN,1225.728,332.300705,6.160732,31251.848549,XX YY,29.356560,195.359850,2011-07-19,2011-07-19,"observatory, ALMA",GRB110715A,NaN,6.126660,0.581143,NaN,uid://A002/X24c117/Xe4,1,Proof of Concept of Response to Targets of Opp...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00001.E,ALMA engineering data release.
2,2011.0.00001.E,GRB110715A,237.683625,-46.235833,7,0.248443,"[349.02..351.01GHz, 31250.00kHz, 6.41mJy/beam@...",2018-07-10,2,0.526604,26.540696,12m,NaN,6.001451,16.877382,NaN,NaN,1225.728,332.300705,6.160732,31251.848549,XX YY,29.356560,195.359850,2011-07-19,2011-07-19,"observatory, ALMA",GRB110715A,NaN,6.126660,0.581143,NaN,uid://A002/X24c117/Xe4,1,Proof of Concept of Response to Targets of Opp...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00001.E,ALMA engineering data release.
3,2011.0.00001.E,GRB110715A,237.683625,-46.235833,7,0.248443,"[351.02..353.01GHz, 31250.00kHz, 7.66mJy/beam@...",2018-07-10,2,0.526604,26.540696,12m,NaN,6.001451,16.877382,NaN,NaN,1225.728,332.300705,6.160732,31251.848549,XX YY,29.356560,195.359850,2011-07-19,2011-07-19,"observatory, ALMA",GRB110715A,NaN,6.126660,0.581143,NaN,uid://A002/X24c117/Xe4,1,Proof of Concept of Response to Targets of Opp...,E,NaN,TARGET WVR,PASS,NaN,2011.0.00001.E,ALMA engineering data release.
4,2011.0.00001.SV,TW Hya,165.466276,-34.704728,7,0.368748,"[343.51..343.98GHz, 244.14kHz, 5.17mJy/beam@10...",2016-06-24,8,1.670760,0.204330,12m,NaN,10.917078,16.595796,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",7106.400,278.678757,22.954625,244.157214,XX YY,18.668068,59.099155,2011-04-22,2011-04-22,"observatory, ALMA","TW Hya: Band 7: CO, HCO+, HC15N",NaN,4.512420,1.323529,NaN,uid://A002/X1d4c10/X53,3,Science verification observation of TW Hya,SV,NaN,TARGET,PASS,NaN,2011.0.00001.SV,Science Verification (SV) is the process by wh...
5,2011.0.00001.SV,TW Hya,165.466276,-34.704728,7,0.368748,"[345.59..346.06GHz, 244.14kHz, 4.55mJy/beam@10...",2016-06-24,8,1.670760,0.204330,12m,NaN,10.917078,16.595796,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",7106.400,278.678757,22.954625,244.157214,XX YY,18.668068,59.099155,2011-04-22,2011-04-22,"observatory, ALMA","TW Hya: Band 7: CO, HCO+, HC15N",NaN,4.512420,1.323529,NaN,uid://A002/X1d4c10/X53,3,Science verification observation of TW Hya,SV,NaN,TARGET,PASS,NaN,2011.0.00001.SV,Science Verification (SV) is the process by wh...
6,2011.0.00001.SV,TW Hya,165.466276,-34.704728,7,0.368748,"[356.52..356.99GHz, 244.14kHz, 4.52mJy/beam@10...",2016-06-24,8,1.670760,0.204330,12m,NaN,10.917078,16.595796,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",7106.400,278.678757,22.954625,244.157214,XX YY,18.668068,59.099155,2011-04-22,2011-04-22,"observatory, ALMA","TW Hya: Band 7: CO, HCO+, HC15N",NaN,4.512420,1.323529,NaN,uid://A002/X1d4c10/X53,3,Science verification observation of TW Hya,SV,NaN,TARGET,PASS,NaN,

### Make columns from Frequency support lists so we can use them easier

In [23]:
freq_supp_cols = pd.DataFrame(full['Frequency support'].to_list())
fsc_names = ['Frequency', 'Resolution', 'Sensitivity', 'Sensitivity Native', 'Pol. Product', 'FS Type']
freq_supp_cols.columns = fsc_names
freq_supp_cols.head()

,Frequency,Resolution,Sensitivity,Sensitivity Native,Pol. Product,FS Type
0,337.02..339.01GHz,31250.00kHz,6.33mJy/beam@10km/s,0.48uJy/beam@native,XX YY,continuum
1,338.96..340.94GHz,31250.00kHz,6.13mJy/beam@10km/s,0.46uJy/beam@native,XX YY,continuum
2,349.02..351.01GHz,31250.00kHz,6.41mJy/beam@10km/s,0.49uJy/beam@native,XX YY,continuum
3,351.02..353.01GHz,31250.00kHz,7.66mJy/beam@10km/s,0.59uJy/beam@native,XX YY,continuum
4,343.51..343.98GHz,244.14kHz,5.17mJy/beam@10km/s,0.81uJy/beam@native,XX YY,line


### Drop "unneccessary" columns from Frequency Support

In [24]:
freq_supp_cols = freq_supp_cols[['Frequency', 'FS Type']]
freq_supp_cols.head()

,Frequency,FS Type
0,337.02..339.01GHz,continuum
1,338.96..340.94GHz,continuum
2,349.02..351.01GHz,continuum
3,351.02..353.01GHz,continuum
4,343.51..343.98GHz,line


### Select relevant columns to make final 'data' data frame

In [25]:
data = full[['Project code', 'Scientific category', 'Science keyword', 'Project title', 'Type', 'Scan intent', 'proposal_abstract']]
data = data.join(freq_supp_cols)
data

,Project code,Scientific category,Science keyword,Project title,Type,Scan intent,proposal_abstract,Frequency,FS Type
0,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,337.02..339.01GHz,continuum
1,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,338.96..340.94GHz,continuum
2,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,349.02..351.01GHz,continuum
3,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,351.02..353.01GHz,continuum
4,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,343.51..343.98GHz,line
5,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,345.59..346.06GHz,line
6,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,356.52..356.99GHz,line
7,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,357.76..358.23GHz,line
8,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,335.00..336.98GHz,continuum
...,...,...,...,...,...,...,...,...,...


## Break Frequency column into low and high columns

### Check that all measurement frequencies are in GHz before dropping from Frequency strings

In [26]:
data.shape

(307352, 9)

In [27]:
data[data['Frequency'].str.contains('GHz')==True]

,Project code,Scientific category,Science keyword,Project title,Type,Scan intent,proposal_abstract,Frequency,FS Type
0,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,337.02..339.01GHz,continuum
1,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,338.96..340.94GHz,continuum
2,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,349.02..351.01GHz,continuum
3,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,351.02..353.01GHz,continuum
4,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,343.51..343.98GHz,line
5,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,345.59..346.06GHz,line
6,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,356.52..356.99GHz,line
7,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,357.76..358.23GHz,line
8,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,335.00..336.98GHz,continuum
...,...,...,...,...,...,...,...,...,...


### Remove GHz from Frequency string

In [28]:
data['Frequency'] = data['Frequency'].str.replace('GHz', '')
data.head()

,Project code,Scientific category,Science keyword,Project title,Type,Scan intent,proposal_abstract,Frequency,FS Type
0,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,337.02..339.01,continuum
1,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,338.96..340.94,continuum
2,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,349.02..351.01,continuum
3,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,351.02..353.01,continuum
4,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,343.51..343.98,line


### Split Frequency on '..'

In [29]:
data[['low_freq','high_freq']] = data['Frequency'].str.split(r'\.\.', n=1, regex=True, expand=True)

In [30]:
data.head()

,Project code,Scientific category,Science keyword,Project title,Type,Scan intent,proposal_abstract,Frequency,FS Type,low_freq,high_freq
0,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,337.02..339.01,continuum,337.02,339.01
1,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,338.96..340.94,continuum,338.96,340.94
2,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,349.02..351.01,continuum,349.02,351.01
3,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,351.02..353.01,continuum,351.02,353.01
4,2011.0.00001.SV,Disks and planet formation,"Disks around low-mass stars, Astrochemistry",Science verification observation of TW Hya,SV,TARGET,Science Verification (SV) is the process by wh...,343.51..343.98,line,343.51,343.98


### Convert new columns to float types

In [31]:
data[['low_freq', 'high_freq']] = data[['low_freq', 'high_freq']].astype('float')

## Create measurement, observation, and project tables

### Check out the Type column and check if there is correlation with line/continuum type

In [32]:
data.loc[data['Type'] == 'E']

,Project code,Scientific category,Science keyword,Project title,Type,Scan intent,proposal_abstract,Frequency,FS Type,low_freq,high_freq
0,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,337.02..339.01,continuum,337.02,339.01
1,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,338.96..340.94,continuum,338.96,340.94
2,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,349.02..351.01,continuum,349.02,351.01
3,2011.0.00001.E,NaN,NaN,Proof of Concept of Response to Targets of Opp...,E,TARGET WVR,ALMA engineering data release.,351.02..353.01,continuum,351.02,353.01
8,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,335.00..336.98,continuum,335.00,336.98
9,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,336.94..338.92,continuum,336.94,338.92
10,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,347.00..348.98,continuum,347.00,348.98
11,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,349.00..350.98,continuum,349.00,350.98
12,2011.0.00002.E,NaN,NaN,Pluto Band 7 TDM data in support of NASA JPL N...,E,TARGET,ALMA engineering data release.,334.99..336.98,continuum,334.99,336.98
...,...,...,...,...,...,...,...,...,...,...,...


### Count number of line and continuum measurements for each project

In [33]:
small = data[['Project code', 'FS Type']]
small.groupby('Project code').value_counts()

Project code     FS Type  
2011.0.00001.E   continuum     4
2011.0.00001.SV  line          4
2011.0.00002.E   continuum    24
2011.0.00002.SV  continuum     4
2011.0.00003.SV  line          4
2011.0.00004.E   line          4
2011.0.00004.SV  line         22
2011.0.00005.E   continuum    16
2011.0.00005.SV  line          4
2011.0.00006.E   line          6
                 continuum     2
2011.0.00006.SV  continuum     8
                              ..
2023.1.01703.S   line          4
2023.1.01706.S   line          6
2023.1.01707.S   continuum    18
                 line          6
2023.1.01710.S   continuum     9
                 line          3
2023.1.01719.S   continuum     4
2023.1.01720.S   continuum     8
                 line          6
2023.1.01721.S   line          8
2023.A.00003.S   line         20
2023.A.00009.S   line         15
Name: count, Length: 6066, dtype: int64

In [34]:
small['Project code'].value_counts()

Project code
2021.1.00869.L    24164
2018.1.00526.S    12936
2019.1.00195.L    12204
2015.1.01289.S     4824
2019.1.01719.S     4036
2018.1.00489.S     3600
2019.1.01813.S     3528
2016.1.01346.S     2976
2023.1.01099.S     2868
2022.1.00342.S     2600
2019.1.00218.S     2568
2022.1.01203.S     2490
                  ...  
2013.1.00914.S        2
2017.1.01057.S        2
2016.1.01265.S        2
2013.1.00568.S        2
2013.1.00659.S        2
2015.1.00793.S        2
2017.1.01230.S        1
2021.1.01556.S        1
2017.1.01659.S        1
2015.1.01359.S        1
2012.1.00797.S        1
2015.1.01107.S        1
Name: count, Length: 4552, dtype: int64

In [35]:
grouped = small.groupby('Project code').value_counts().to_frame().reset_index()
grouped

,Project code,FS Type,count
0,2011.0.00001.E,continuum,4
1,2011.0.00001.SV,line,4
2,2011.0.00002.E,continuum,24
3,2011.0.00002.SV,continuum,4
4,2011.0.00003.SV,line,4
5,2011.0.00004.E,line,4
6,2011.0.00004.SV,line,22
7,2011.0.00005.E,continuum,16
8,2011.0.00005.SV,line,4
...,...,...,...


In [36]:
test = grouped.loc[(grouped['FS Type'] == 'line')]
test

,Project code,FS Type,count
1,2011.0.00001.SV,line,4
4,2011.0.00003.SV,line,4
5,2011.0.00004.E,line,4
6,2011.0.00004.SV,line,22
8,2011.0.00005.SV,line,4
9,2011.0.00006.E,line,6
12,2011.0.00007.SV,line,8
14,2011.0.00008.SV,line,4
15,2011.0.00009.SV,line,20
...,...,...,...


### Create projects table

In [37]:
projects = data[['Project code', 'Project title', 'Type', 'proposal_abstract', 'FS Type']]
projects

,Project code,Project title,Type,proposal_abstract,FS Type
0,2011.0.00001.E,Proof of Concept of Response to Targets of Opp...,E,ALMA engineering data release.,continuum
1,2011.0.00001.E,Proof of Concept of Response to Targets of Opp...,E,ALMA engineering data release.,continuum
2,2011.0.00001.E,Proof of Concept of Response to Targets of Opp...,E,ALMA engineering data release.,continuum
3,2011.0.00001.E,Proof of Concept of Response to Targets of Opp...,E,ALMA engineering data release.,continuum
4,2011.0.00001.SV,Science verification observation of TW Hya,SV,Science Verification (SV) is the process by wh...,line
5,2011.0.00001.SV,Science verification observation of TW Hya,SV,Science Verification (SV) is the process by wh...,line
6,2011.0.00001.SV,Science verification observation of TW Hya,SV,Science Verification (SV) is the process by wh...,line
7,2011.0.00001.SV,Science verification observation of TW Hya,SV,Science Verification (SV) is the process by wh...,line
8,2011.0.00002.E,Pluto Band 7 TDM data in support of NASA JPL N...,E,ALMA engineering data release.,continuum
...,...,...,...,...,...


In [38]:
projects = projects.drop_duplicates()
projects = projects.reset_index()
projects = projects.drop('index', axis=1)
projects

,Project code,Project title,Type,proposal_abstract,FS Type
0,2011.0.00001.E,Proof of Concept of Response to Targets of Opp...,E,ALMA engineering data release.,continuum
1,2011.0.00001.SV,Science verification observation of TW Hya,SV,Science Verification (SV) is the process by wh...,line
2,2011.0.00002.E,Pluto Band 7 TDM data in support of NASA JPL N...,E,ALMA engineering data release.,continuum
3,2011.0.00002.SV,Science verification observation of NGC3256,SV,Science Verification (SV) is the process by wh...,continuum
4,2011.0.00003.SV,Science verification observation of Antennae g...,SV,Science Verification (SV) is the process by wh...,line
5,2011.0.00004.E,Vela Pulsar J0835-4510,E,ALMA engineering data release.,line
6,2011.0.00004.SV,Science verification observation of M100,SV,Science Verification (SV) is the process by wh...,line
7,2011.0.00005.E,HL Tau Band 9 B2B,E,ALMA engineering data release.,continuum
8,2011.0.00005.SV,Science verification observation of SgrA*,SV,Science Verification (SV) is the process by wh...,line
...,...,...,...,...,...


### Get project line/continuum classification by sorting by FS Type and dropping duplicates
This works because we sort projects that have any `line` measurements to the top and keep that row of the project classification.

Thus any continuum only projects get sorted to the bottom and are classified as line if they're associated with a line above it.

In [39]:
projects = projects.sort_values(by='FS Type', ascending=False)
projects = projects.drop_duplicates(subset=['Project code', 'Project title', 'Type', 'proposal_abstract'])
projects = projects.reset_index()
projects = projects.drop('index', axis=1)
projects['FS Type'].value_counts()

FS Type
line         3644
continuum     908
Name: count, dtype: int64

### Look at high-level `Type` and get counts to understand how many projects we drop using type

In [40]:
projects = projects.sort_values('Type', ascending=True)
projects['Type'].value_counts()

Type
S      4352
T       102
V        46
L        27
SV       17
E         6
CAL       1
P         1
Name: count, dtype: int64

### From review of Type, we'll exclude (CAL, SV, E)

In [41]:
projects = projects[~projects['Type'].isin(['CAL', 'SV', 'E'])]
projects = projects.reset_index()
projects = projects.drop(['index', 'Type'], axis=1)

### Rename columns

In [42]:
projects = projects.rename(columns={'Project code':'project_code',
                         'Project title':'project_title',
                         'proposal_abstract':'project_abstract',
                         'FS Type':'fs_type'}
)
projects

,project_code,project_title,project_abstract,fs_type
0,2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line
1,2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line
2,2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line
3,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line
4,2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line
5,2022.1.00338.L,The ALMA survey to Resolve exoKuiper belt Subs...,ExoKuiper belts are a ubiquitous component of ...,line
6,2023.1.00360.L,UNveiling the Initial Conditions of high-mass ...,High-mass stars play a key role in shaping the...,line
7,2019.1.01634.L,REBELS: An ALMA Large Program to Discover the ...,REBELS will construct the first large statisti...,line
8,2021.1.00172.L,ACES: The ALMA CMZ Exploration Survey,The extreme environment and relative proximity...,line
...,...,...,...,...


### Create target column (binary classification line=1, continuum=0)

In [43]:
projects['target'] = 1
projects.loc[projects['fs_type'] == 'continuum', 'target'] = 0

### Check that this operation worked

In [44]:
projects.query('fs_type == "continuum"').target.sum()

0

In [45]:
projects.query('fs_type == "line"').target.sum() == projects.query('fs_type == "line"').shape[0]

True

## Basic text processing of title and abstract
### Create combined title-abstract column

In [46]:
projects['raw_text'] = projects.project_title.str.strip() + '. ' + projects.project_abstract.str.strip()
projects.head()

,project_code,project_title,project_abstract,fs_type,target,raw_text
0,2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1,Fifty AU STudy of the chemistry in the disk/en...
1,2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1,COMPASS: Complex Organic Molecules in Protosta...
2,2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1,ALCHEMI: the ALMA Comprehensive High-resolutio...
3,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...
4,2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1,Bulge symmetry or not? The hidden dynamics of ...


In [55]:
projects.raw_text.apply(lambda x: preprocess(x))

0       fifty au study of the chemistry in the disk en...
1       compass complex organic molecules in protostar...
2       alchemi the alma comprehensive high resolution...
3       alma jelly survey of nearby jellyfish and ram ...
4       bulge symmetry or not the hidden dynamics of t...
5       the alma survey to resolve exokuiper belt subs...
6       unveiling the initial conditions of high mass ...
7       rebels an alma large program to discover the m...
8       aces the alma cmz exploration survey the extre...
9       the cosmos high z alma miri population survey ...
10      cristal a survey of gas dust and stars on kilo...
11      atomium alma tracing the origins of molecules ...
                              ...                        
4516    the light day scale structure of an extragalac...
4517    connecting the black hole shadow and jet base ...
4518    imaging the global accretion and outflow of sg...
4519    imaging the shadow of a supermassive black hol...
4520    probin

In [59]:
projects['standardized_text'] = projects.raw_text.apply(lambda x: preprocess(x))
projects['no_sw_text'] = projects.standardized_text.apply(lambda x: stopword(x))
projects['lemmatized_sw_text'] = projects.standardized_text.apply(lambda x: lemmatizer(x))
projects['lemmatized_no_sw_text'] = projects.no_sw_text.apply(lambda x: lemmatizer(x))

In [60]:
projects.head()

,project_code,project_title,project_abstract,fs_type,target,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
0,2018.1.01205.L,Fifty AU STudy of the chemistry in the disk/en...,The huge variety of planetary systems discover...,line,1,Fifty AU STudy of the chemistry in the disk/en...,fifty au study of the chemistry in the disk en...,fifty au study chemistry disk envelope system ...,fifty au study of the chemistry in the disk en...,fifty au study chemistry disk envelope system ...
1,2022.1.00316.L,COMPASS: Complex Organic Molecules in Protosta...,The emergence of complex organic molecules in ...,line,1,COMPASS: Complex Organic Molecules in Protosta...,compass complex organic molecules in protostar...,compass complex organic molecules protostars a...,compass complex organic molecule in protostars...,compass complex organic molecule protostars al...
2,2017.1.00161.L,ALCHEMI: the ALMA Comprehensive High-resolutio...,A great variety in gas composition is observed...,line,1,ALCHEMI: the ALMA Comprehensive High-resolutio...,alchemi the alma comprehensive high resolution...,alchemi alma comprehensive high resolution ext...,alchemi the alma comprehensive high resolution...,alchemi alma comprehensive high resolution ext...
3,2021.1.01616.L,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,We propose the first ever statistical survey o...,line,1,ALMA JELLY - Survey of Nearby Jellyfish and Ra...,alma jelly survey of nearby jellyfish and ram ...,alma jelly survey nearby jellyfish ram pressur...,alma jelly survey of nearby jellyfish and ram ...,alma jelly survey nearby jellyfish ram pressur...
4,2021.1.00869.L,Bulge symmetry or not? The hidden dynamics of ...,A radio survey of red giant SiO sources in the...,line,1,Bulge symmetry or not? The hidden dynamics of ...,bulge symmetry or not the hidden dynamics of t...,bulge symmetry hidden dynamics far side radio ...,bulge symmetry or not the hidden dynamic of th...,bulge symmetry hidden dynamic far side radio s...


## Build barebones CSV with code, title, abstract, and type

In [62]:
projects.to_csv('nrao_projects.csv', index=False)

## Create measurements table

In [63]:
measurements = data[['Project code', 'Type', 'Project title', 'proposal_abstract', 'FS Type', 'low_freq', 'high_freq']]
measurements.head()

,Project code,Type,Project title,proposal_abstract,FS Type,low_freq,high_freq
0,2011.0.00001.E,E,Proof of Concept of Response to Targets of Opp...,ALMA engineering data release.,continuum,337.02,339.01
1,2011.0.00001.E,E,Proof of Concept of Response to Targets of Opp...,ALMA engineering data release.,continuum,338.96,340.94
2,2011.0.00001.E,E,Proof of Concept of Response to Targets of Opp...,ALMA engineering data release.,continuum,349.02,351.01
3,2011.0.00001.E,E,Proof of Concept of Response to Targets of Opp...,ALMA engineering data release.,continuum,351.02,353.01
4,2011.0.00001.SV,SV,Science verification observation of TW Hya,Science Verification (SV) is the process by wh...,line,343.51,343.98


### From review of Type, we'll exclude (CAL, SV, E)

In [64]:
measurements = measurements[~measurements['Type'].isin(['CAL', 'SV', 'E'])]
measurements = measurements.reset_index()
measurements = measurements.drop(['index', 'Type'], axis=1)

### Rename columns

In [65]:
measurements = measurements.rename(columns={'Project code':'project_code',
                         'Project title':'project_title',
                         'proposal_abstract':'project_abstract',
                         'FS Type':'fs_type'}
)
measurements.head()

,project_code,project_title,project_abstract,fs_type,low_freq,high_freq
0,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62
1,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93
2,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92
3,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12
4,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53


In [66]:
measurements['target'] = 1
measurements.loc[measurements['fs_type'] == 'continuum', 'target'] = 0

In [67]:
measurements.head()

,project_code,project_title,project_abstract,fs_type,low_freq,high_freq,target
0,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,1
1,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,1
2,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,1
3,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,1
4,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,1


### Create frequency median and frequency difference columns

In [68]:
measurements['diff_freq'] = ((measurements['high_freq'] - measurements['low_freq']))
measurements['med_freq'] = ((measurements['low_freq']) + measurements['diff_freq']/2)
measurements.head()

,project_code,project_title,project_abstract,fs_type,low_freq,high_freq,target,diff_freq,med_freq
0,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,1,0.24,90.500
1,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,1,0.23,90.815
2,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,1,0.23,91.805
3,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,1,0.23,93.005
4,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,1,0.94,218.060


In [69]:
measurements.shape

(307139, 9)

### Inspection of single project

In [70]:
proj1 = measurements[measurements['project_code'] == '2019.1.00193.S']
proj1

,project_code,project_title,project_abstract,fs_type,low_freq,high_freq,target,diff_freq,med_freq
155116,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,290.22,290.28,1,0.06,290.250
155117,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,290.24,290.29,1,0.05,290.265
155118,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,290.28,290.34,1,0.06,290.310
155119,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,290.35,290.41,1,0.06,290.380
155120,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,291.21,291.27,1,0.06,291.240
155121,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,291.35,291.41,1,0.06,291.380
155122,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,291.36,291.41,1,0.05,291.385
155123,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,291.92,291.98,1,0.06,291.950
155124,2019.1.00193.S,Astrochemical confirmation of a circumplanetar...,ALMA observations of the dust and gas in proto...,line,301.23,301.35,1,0.12,301.290
...,...,...,...,...,...,...,...,...,...


### Note, frequency values repeat for this project

In [71]:
proj1.low_freq.value_counts()

low_freq
290.22    2
290.24    2
290.28    2
290.35    2
291.21    2
291.35    2
291.36    2
291.92    2
301.23    2
303.90    2
304.05    2
Name: count, dtype: int64

In [72]:
proj1.high_freq.value_counts()

high_freq
291.41    4
290.28    2
290.29    2
290.34    2
290.41    2
291.27    2
291.98    2
301.35    2
303.96    2
304.11    2
Name: count, dtype: int64

## Remove duplicates

In [73]:
pre_drop = measurements.shape[0]
pre_drop

307139

In [74]:
measurements = measurements.drop_duplicates()
post_drop = measurements.shape[0]
post_drop

67435

In [75]:
pre_drop - post_drop

239704

### Add processed text from projects table to measurements table

In [80]:
measurements = measurements.merge(projects[['project_code',
                                     'raw_text',
                                     'standardized_text',
                                     'no_sw_text', 
                                     'lemmatized_sw_text',
                                     'lemmatized_no_sw_text']], how='inner', on='project_code')
measurements.head(10)

,project_code,project_title,project_abstract,fs_type,low_freq,high_freq,target,diff_freq,med_freq,raw_text,standardized_text,no_sw_text,lemmatized_sw_text,lemmatized_no_sw_text
0,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.38,90.62,1,0.24,90.500,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
1,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,90.70,90.93,1,0.23,90.815,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
2,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,91.69,91.92,1,0.23,91.805,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
3,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,92.89,93.12,1,0.23,93.005,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
4,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,217.59,218.53,1,0.94,218.060,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
5,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,218.67,219.60,1,0.93,219.135,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
6,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,218.91,219.85,1,0.94,219.380,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
7,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,219.52,220.46,1,0.94,219.990,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
8,2011.0.00010.S,The Physics and Chemisty of Gas in Centaurus A...,Centaurus A with its host NGC5128 is the most ...,line,87.05,87.29,1,0.24,87.170,The Physics and Chemisty of Gas in Centaurus A...,the physics and chemisty of gas in centaurus a...,physics chemisty gas centaurus host v centauru...,the physic and chemisty of gas in centaurus a ...,physic chemisty gas centaurus host v centaurus...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


### Generate measurements CSV
#### Currently too large for GIT

In [82]:
filename = 'nrao_measurements'
compression_options = dict(method='zip', archive_name=f'{filename}.csv')
measurements.to_csv(f'{filename}.zip', index=False, compression=compression_options)